In [ ]:
import fasttext

# Load the pre-trained language identification model
model_path = '/path/to/pretrained_model.bin'  # Replace with the path to your downloaded model
model = fasttext.load_model(model_path)

# Text for language identification
text = "Вас с морем соли"

# Predict the language
predicted_language = model.predict(text, k=1)  # k=1 means only top 1 prediction will be returned
language_code = predicted_language[0][0].split('__label__')[1]

print("Predicted Language Code:", language_code)
